# Import

In [1]:
import pandas as pd
import re
import ast
import json
import statistics
import openai
from multiprocessing import Pool
openai.api_key = "YOUR API KEY"

In [2]:
import os
import sys
sys.path.insert(1, '../')
from utils.prompting import *
from utils.postprocessing import *
from utils.comment_kp_matching import *

# Read Predicted Summary

In [3]:
qqsum_output_path = "../output/atlas-xl-seed2-lgret-lglm"

In [4]:
ground_truth_df = pd.read_pickle("../data/test/test.pkl")
ground_truth_df = ground_truth_df.drop(columns=['reviews', 'reviewText', 'title', 'fact_check_article'])
ground_truth_df['retrieved_relevant_sent_len'] = ground_truth_df['retrieved_relevant_sent'].str.len()

predicted_df = pd.read_json(qqsum_output_path + "/test-result.jsonl", lines=True)
predicted_df['passages'] = predicted_df['passages'].apply(lambda x: [pas['text'] for pas in x])
predicted_df['passages_len'] = predicted_df['passages'].str.len()
predicted_df = predicted_df[['query', 'generation', 'passages', 'passages_scores', 'passages_len', 'comment_clusters', 'id']]
df = ground_truth_df.merge(predicted_df, on=['query', 'id'])

In [5]:
# df = df[df['passages'].str.len() > 0]
# df['passages_scores_min_thres_ref_ground'] = df.apply(lambda row: row['passages_scores'][:len(row['retrieved_relevant_sent'])][-1], axis=1)
# df['passages_scores_min_thres_ref_ground'].mean()

In [6]:
df['summary'] = df['generation'].apply(lambda x: re.findall("\[\/INST\] *((.+\n*)+)$", x)[0][0].replace("</s>", ""))
df['summary'] = df['summary'].apply(lambda x: re.sub("(Therefore|Thus)(.+\n*)+$", "", x))
df['final_summary'] = df['summary'].apply(lambda x: re.findall("(While[^\n]+\n+(\+ *[0-9]+[^\n]+\n*)+)", x))
df['final_summary'] = df['final_summary'].apply(lambda x: [e[0] for e in x])
df['final_summary_text'] = df['final_summary'].apply(lambda x: "\n\n".join(x[:1]))

## Post-process summary into KPs

### Trial

In [7]:
row = df.iloc[0]
print(row['id'])
get_kp_from_summary(row['final_summary_text'])

3609


'```json\n[{"key_point": "the old key components need to be taken to a locksmith or dealership", "prevalence": "7"}]\n```'

### Run

In [8]:
from utils.postprocessing import *

In [9]:
df['my_category'] = 1
num_workers = 1

In [10]:
inputs = [(qqsum_output_path + "/post_processed_cache",
# inputs = [(qqsum_output_path + "/post_processed_cache/rd",
           domain,
           df[df['my_category'] == domain].reset_index(drop=True)
           )
          for domain in df['my_category'].unique()]

In [11]:
start_time = time.time()
with Pool(num_workers) as processor:
    data = processor.starmap(prompted_claim_split_generation, inputs)
print("TIME ELAPSED", time.time() - start_time)

1 :  Loaded saved file. Done
TIME ELAPSED 0.15581655502319336


In [12]:
processed_df = pd.concat(data)
processed_df = processed_df[processed_df['comment_clusters'].str.len() > 0]
processed_df = processed_df[processed_df['final_summary'].str.len() > 0]

In [13]:
processed_df['claim_split_predicted'] = processed_df['claim_split_predicted'].apply(lambda x: re.sub(r"\n+ *", "", x.replace("json", "").replace("`", "")))
mask = processed_df['claim_split_predicted'].apply(lambda x: len(re.findall(r"(: *)\'((?:[^':]*\'+[^':,]*)+)\'( *)", x, re.DOTALL)) > 0)
processed_df.loc[mask, 'claim_split_predicted'] = processed_df.loc[mask, 'claim_split_predicted'].apply(
    lambda x: re.sub(r"(: *)\'((?:[^':]*\'+[^':]*)+)\'( *,)", r'\1"""\2"""\3', x))
processed_df['claim_split_predicted'] = processed_df['claim_split_predicted'].apply(extract_claims)

In [14]:
processed_df = processed_df.reset_index(drop=True)

In [15]:
processed_df = processed_df.apply(match_claim_with_cluster, axis=1)

In [16]:
claim_split_predicted = pd.json_normalize(
    processed_df.to_dict(orient='records'), 
    "matching_comment_clusters", ["id", "query", "passages", 'passages_len']
)
claim_split_predicted['prevalence'] = claim_split_predicted['prevalence'].astype(int)
claim_split_predicted

,comments,cluster_size,key_point,prevalence,id,query,passages,passages_len
0,[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,7
1,"[In order to use this key, you will need two w...",17,you need two originally programmed keys to pro...,17,2799,I have a base model 2007 F150 XL V6. I would l...,"[In order to use this key, you will need two w...",22
2,[I bought my 2003 Ford F250 truck from a used ...,5,it's possible to use a remake key or a key fro...,5,2799,I have a base model 2007 F150 XL V6. I would l...,"[In order to use this key, you will need two w...",22
3,[It works best if you use a detailer as a lubr...,3,using a detailer as a lubricant,3,2501,"Do you need to use a lubricant? If so, what do...",[It works best if you use a detailer as a lubr...,3
4,"[The cup-holders work great and adjust easily,...",16,the adjustable cup holders are easy to use and...,16,3250,How sturdy are the adjustable cup holders? Wou...,"[The cup-holders work great and adjust easily,...",16
...,...,...,...,...,...,...,...,...
109,[I will admit that I missed the slim look of t...,8,the 3DS is backwards compatible with DS games,8,539,Can This replace my DS lite? Will I still be a...,[I will admit that I missed the slim look of t...,8
110,[inFamous Second Son however doesn't keep you ...,16,Infamous Second Son is a bigger and more refin...,16,1837,I've never played Infamous 1 - Should you find...,[inFamous Second Son however doesn't keep you ...,20
111,"[In fact, before playing I was very skeptical ...",4,Infamous Second Son is a game that surpassed t...,4,1837,I've never played Infamous 1 - Should you find...,[inFamous Second Son however doesn't keep you ...,20
112,"[In terms of controls, the game allows you wit...",9,the game allows the use of Wii mote only,9,2189,do you have to use the remote and nunchuck? or...,"[In terms of controls, the game allows you wit...",9


In [17]:
evaluation_df = claim_split_predicted.explode(['passages'])
evaluation_df

,comments,cluster_size,key_point,prevalence,id,query,passages,passages_len
0,[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,[the plastic case of my old keys was broken an...,7
0,[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,The only issue I had with my old key was that ...,7
0,[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,"Like it said in the description, you need to p...",7
0,[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,"Either way, I Broke 2 original car keys.",7
0,[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,Once i put everything just like the old key an...,7
...,...,...,...,...,...,...,...,...
112,"[In terms of controls, the game allows you wit...",9,the game allows the use of Wii mote only,9,2189,do you have to use the remote and nunchuck? or...,Like others have mentioned though the wii remo...,9
113,"[If you want to use it on a tablet or phone, y...",4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,"If you want to use it on a tablet or phone, yo...",4
113,"[If you want to use it on a tablet or phone, y...",4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,"I bought this so I could have a small, portabl...",4
113,"[If you want to use it on a tablet or phone, y...",4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,Played Sonic cd and it was great with this con...,4


## KP Matching Annotation

### Trial

In [18]:
row = evaluation_df.iloc[20]
print(row['id'])
print("Question:", row['query'])
print("Comment:", row['passages'])
print("Key Point:", row['key_point'])
response = get_comment_kp_annotation_completion(row['query'], row['key_point'], row['passages'])
print()
print("GPT4-o-mini Annotation:\n", response)

2799
Question: I have a base model 2007 F150 XL V6. I would like to purchase this key but wanted to know if I would need 2 programmed keys in order for this to work?
Comment: I bought my 2003 Ford F250 truck from a used car dealer and the only key he had was a remake, not the original.
Key Point: you need two originally programmed keys to program a new one

GPT4-o-mini Annotation:
 To determine if the key point matches the review sentence, we need to analyze both the key point and the review sentence in relation to the query.

The key point states: "you need two originally programmed keys to program a new one." This indicates that in order to program a new key for a vehicle, the owner must have two original keys that were programmed for that vehicle.

The review sentence states: "I bought my 2003 Ford F250 truck from a used car dealer and the only key he had was a remake, not the original." This sentence indicates that the reviewer only received a remake key and did not have the origin

### Run

In [19]:
from utils.postprocessing import *

In [20]:
df['my_category'] = 1
num_workers = 1

In [21]:
inputs = [(qqsum_output_path + "/experiment_comment_kp_matching_cache",
           domain,
           df[df['my_category'] == domain].reset_index(drop=True)
           )
          for domain in df['my_category'].unique()]

In [22]:
start_time = time.time()
with Pool(num_workers) as processor:
    data = processor.starmap(prompted_claim_split_generation, inputs)
print("TIME ELAPSED", time.time() - start_time)

1 :  Loaded saved file. Done
TIME ELAPSED 0.1025540828704834


In [23]:
processed_evaluation_df = pd.concat(data)
processed_evaluation_df = processed_evaluation_df.drop(columns=['passages_len'])
processed_evaluation_df

,matching_response,comments,cluster_size,key_point,prevalence,id,query,passages,my_category
0,"The key point states that ""the old key compone...",[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,[the plastic case of my old keys was broken an...,1
1,"The key point states that ""the old key compone...",[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,The only issue I had with my old key was that ...,1
2,"The key point states that ""the old key compone...",[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,"Like it said in the description, you need to p...",1
3,"The key point states that ""the old key compone...",[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,"Either way, I Broke 2 original car keys.",1
4,To determine if the key point matches the revi...,[[the plastic case of my old keys was broken a...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,Once i put everything just like the old key an...,1
...,...,...,...,...,...,...,...,...,...
3013,To determine if the key point matches the revi...,"[In terms of controls, the game allows you wit...",9,the game allows the use of Wii mote only,9,2189,do you have to use the remote and nunchuck? or...,Like others have mentioned though the wii remo...,1
3014,The key point states that the controller is po...,"[If you want to use it on a tablet or phone, y...",4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,"If you want to use it on a tablet or phone, yo...",1
3015,The key point states that the controller is po...,"[If you want to use it on a tablet or phone, y...",4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,"I bought this so I could have a small, portabl...",1
3016,To determine if the key point matches the revi...,"[If you want to use it on a tablet or phone, y...",4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,Played Sonic cd and it was great with this con...,1


In [24]:
def entailment_detail_labels(response_text):
    if "not at all" in response_text.lower():
        return "not at all"
    elif "somewhat not well" in response_text.lower():
        return "somewhat not well"
    elif "somewhat well" in response_text.lower():
        return "somewhat well"
    elif "very well" in response_text.lower():
        return "very well"

In [25]:
def entailment_labels(response_text):
    if "not at all" in response_text.lower():
        return 0
    elif "somewhat not well" in response_text.lower():
        return 0
    elif "somewhat well" in response_text.lower():
        return 1
    elif "very well" in response_text.lower():
        return 1

In [26]:
processed_evaluation_df['matching_detail_label'] = processed_evaluation_df['matching_response'].apply(entailment_detail_labels)
processed_evaluation_df['matching_label'] = processed_evaluation_df['matching_response'].apply(entailment_labels)

# KP-Comment Matching

## Precision

In [27]:
evaluation_df = claim_split_predicted.explode(['comments'])
evaluation_df

,comments,cluster_size,key_point,prevalence,id,query,passages,passages_len
0,[the plastic case of my old keys was broken an...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,7
0,The only issue I had with my old key was that ...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,7
0,"Like it said in the description, you need to p...",7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,7
0,"Either way, I Broke 2 original car keys.",7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,7
0,Once i put everything just like the old key an...,7,the old key components need to be taken to a l...,7,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,7
...,...,...,...,...,...,...,...,...
112,Like others have mentioned though the wii remo...,9,the game allows the use of Wii mote only,9,2189,do you have to use the remote and nunchuck? or...,"[In terms of controls, the game allows you wit...",9
113,"If you want to use it on a tablet or phone, yo...",4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,"[If you want to use it on a tablet or phone, y...",4
113,"I bought this so I could have a small, portabl...",4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,"[If you want to use it on a tablet or phone, y...",4
113,Played Sonic cd and it was great with this con...,4,this controller is portable and works well wit...,4,3049,hi this controller work with emulators run on ...,"[If you want to use it on a tablet or phone, y...",4


In [28]:
ref_df = processed_evaluation_df[['id', 'query', 'my_category', 'key_point', 'passages', 'matching_detail_label', 'matching_label']]
ref_df = ref_df.rename(columns={'passages': 'comments'})

In [29]:
precision_df = evaluation_df.merge(ref_df.drop_duplicates(['id', 'query', 'my_category', 'key_point', 'comments']), how='left')

In [30]:
precision_df.shape

(1940, 11)

In [31]:
precision_df['matching_label'].value_counts(normalize=True)

matching_label
1.0    0.646209
0.0    0.353791
Name: proportion, dtype: float64

In [32]:
def calculate_precision(grp):
    if pd.isna(grp['matching_label'].iloc[0]):
        return None
    
    precision = len(grp[grp['matching_label'] == 1]) / len(grp)
    return pd.DataFrame({'precision': [precision]})

In [33]:
# precision_df = precision_df[precision_df['prevalence'] >= 4]

In [34]:
stat_df = precision_df.groupby(['id']).apply(calculate_precision)
precision = stat_df['precision'].mean()
precision

/tmp/ipykernel_10143/732016646.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_df = precision_df.groupby(['id']).apply(calculate_precision)


0.6939461523031528

## Recall

In [35]:
evaluation_df = claim_split_predicted.explode(['comments'])
evaluation_df['predicted_label'] = 1

In [36]:
mask = processed_evaluation_df['matching_label'] == 1
ref_df = processed_evaluation_df[mask][['id', 'query', 'my_category', 'key_point', 'passages', 'matching_detail_label', 'matching_label']]
ref_df = ref_df.rename(columns={'passages': 'comments'})

In [37]:
evaluation_df = evaluation_df.drop_duplicates(['comments', 'cluster_size', 'key_point', 'prevalence', 'id', 'query'])

In [38]:
recall_df = ref_df.merge(evaluation_df, how='left')

In [39]:
recall_df

,id,query,my_category,key_point,comments,matching_detail_label,matching_label,cluster_size,prevalence,passages,passages_len,predicted_label
0,3609,broke key head --can I just get new key head a...,1,the old key components need to be taken to a l...,[the plastic case of my old keys was broken an...,very well,1.0,7.0,7.0,[[the plastic case of my old keys was broken a...,7,1.0
1,3609,broke key head --can I just get new key head a...,1,the old key components need to be taken to a l...,The only issue I had with my old key was that ...,somewhat well,1.0,7.0,7.0,[[the plastic case of my old keys was broken a...,7,1.0
2,3609,broke key head --can I just get new key head a...,1,the old key components need to be taken to a l...,"Like it said in the description, you need to p...",somewhat well,1.0,7.0,7.0,[[the plastic case of my old keys was broken a...,7,1.0
3,3609,broke key head --can I just get new key head a...,1,the old key components need to be taken to a l...,Once i put everything just like the old key an...,very well,1.0,7.0,7.0,[[the plastic case of my old keys was broken a...,7,1.0
4,3609,broke key head --can I just get new key head a...,1,the old key components need to be taken to a l...,"They want $200 for new key, even though my mot...",somewhat well,1.0,7.0,7.0,[[the plastic case of my old keys was broken a...,7,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1719,2189,do you have to use the remote and nunchuck? or...,1,the game allows the use of Wii mote only,Like others have mentioned though the wii remo...,somewhat well,1.0,9.0,9.0,"[In terms of controls, the game allows you wit...",9,1.0
1720,3049,hi this controller work with emulators run on ...,1,this controller is portable and works well wit...,"If you want to use it on a tablet or phone, yo...",very well,1.0,4.0,4.0,"[If you want to use it on a tablet or phone, y...",4,1.0
1721,3049,hi this controller work with emulators run on ...,1,this controller is portable and works well wit...,"I bought this so I could have a small, portabl...",very well,1.0,4.0,4.0,"[If you want to use it on a tablet or phone, y...",4,1.0
1722,3049,hi this controller work with emulators run on ...,1,this controller is portable and works well wit...,Played Sonic cd and it was great with this con...,somewhat well,1.0,4.0,4.0,"[If you want to use it on a tablet or phone, y...",4,1.0


In [40]:
recall_df[pd.isnull(recall_df['predicted_label'])]

,id,query,my_category,key_point,comments,matching_detail_label,matching_label,cluster_size,prevalence,passages,passages_len,predicted_label
17,2799,I have a base model 2007 F150 XL V6. I would l...,1,you need two originally programmed keys to pro...,I bought my 2003 Ford F250 truck from a used c...,somewhat well,1.0,NaN,NaN,NaN,NaN,NaN
20,2799,I have a base model 2007 F150 XL V6. I would l...,1,you need two originally programmed keys to pro...,"If you don't have two keys already, be prepare...",very well,1.0,NaN,NaN,NaN,NaN,NaN
23,2799,I have a base model 2007 F150 XL V6. I would l...,1,it's possible to use a remake key or a key fro...,"Also, there were no instructions included (the...",somewhat well,1.0,NaN,NaN,NaN,NaN,NaN
24,2799,I have a base model 2007 F150 XL V6. I would l...,1,it's possible to use a remake key or a key fro...,"I already have a backup remote key fob, so if ...",very well,1.0,NaN,NaN,NaN,NaN,NaN
25,2799,I have a base model 2007 F150 XL V6. I would l...,1,it's possible to use a remake key or a key fro...,I had 1 good key and a clone key in my case so...,somewhat well,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1706,1837,I've never played Infamous 1 - Should you find...,1,Infamous Second Son is a game that surpassed t...,"Absolutely a next Gen game, I have played thr...",somewhat well,1.0,NaN,NaN,NaN,NaN,NaN
1707,1837,I've never played Infamous 1 - Should you find...,1,Infamous Second Son is a game that surpassed t...,"Despite the aforementioned shortcomings, inFam...",somewhat well,1.0,NaN,NaN,NaN,NaN,NaN
1710,1837,I've never played Infamous 1 - Should you find...,1,Infamous Second Son is a game that surpassed t...,Im an Infamous fan i love the series and Secon...,very well,1.0,NaN,NaN,NaN,NaN,NaN
1711,1837,I've never played Infamous 1 - Should you find...,1,Infamous Second Son is a game that surpassed t...,I have never played the previous inFAMOUS titl...,somewhat well,1.0,NaN,NaN,NaN,NaN,NaN


In [41]:
recall_df['predicted_label'] = recall_df['predicted_label'].fillna(0)

In [42]:
recall_df['predicted_label'].value_counts(normalize=True)

predicted_label
1.0    0.725638
0.0    0.274362
Name: proportion, dtype: float64

In [43]:
def calculate_recall(grp):
    recall = len(grp[grp['predicted_label'] == 1]) / len(grp)
    return pd.DataFrame({'recall': [recall]})

In [44]:
# recall_df = recall_df[~recall_df['key_point'].isin(low_perf.reset_index()['key_point'])]

In [45]:
stat_df = recall_df.groupby(['id']).apply(calculate_recall)

/tmp/ipykernel_10143/2368992875.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_df = recall_df.groupby(['id']).apply(calculate_recall)


In [46]:
recall = stat_df['recall'].mean()
recall

0.8690587654940435

## F1

In [47]:
F1 = 2 * (precision * recall) / (precision + recall)
F1

0.7716930120602097

## QuantErr

In [48]:
def calculate_precision_prevalence(grp):
    if pd.isna(grp['matching_label'].iloc[0]):
        return None
    return pd.DataFrame({'correct_prevalence': [len(grp[grp['matching_label'] == 1])]})
precision_prevalence = precision_df.groupby(['id', 'key_point'], sort=False).apply(calculate_precision_prevalence).reset_index().drop(columns=['level_2'])
precision_prevalence

/tmp/ipykernel_10143/1931902676.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  precision_prevalence = precision_df.groupby(['id', 'key_point'], sort=False).apply(calculate_precision_prevalence).reset_index().drop(columns=['level_2'])


,id,key_point,correct_prevalence
0,3609,the old key components need to be taken to a l...,5
1,2799,you need two originally programmed keys to pro...,16
2,2799,it's possible to use a remake key or a key fro...,5
3,2501,using a detailer as a lubricant,2
4,3250,the adjustable cup holders are easy to use and...,12
...,...,...,...
109,539,the 3DS is backwards compatible with DS games,6
110,1837,Infamous Second Son is a bigger and more refin...,11
111,1837,Infamous Second Son is a game that surpassed t...,4
112,2189,the game allows the use of Wii mote only,5


In [49]:
def calculate_recall_prevalence(grp):
    return pd.DataFrame({'expected_prevalence': [len(grp[grp['matching_label'] == 1])]})
recall_prevalence = recall_df.groupby(['id', 'key_point'], sort=False).apply(calculate_recall_prevalence).reset_index().drop(columns=['level_2'])
recall_prevalence

/tmp/ipykernel_10143/3022614201.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  recall_prevalence = recall_df.groupby(['id', 'key_point'], sort=False).apply(calculate_recall_prevalence).reset_index().drop(columns=['level_2'])


,id,key_point,expected_prevalence
0,3609,the old key components need to be taken to a l...,5
1,2799,you need two originally programmed keys to pro...,18
2,2799,it's possible to use a remake key or a key fro...,14
3,2501,using a detailer as a lubricant,2
4,3250,the adjustable cup holders are easy to use and...,12
...,...,...,...
109,539,the 3DS is backwards compatible with DS games,6
110,1837,Infamous Second Son is a bigger and more refin...,13
111,1837,Infamous Second Son is a game that surpassed t...,12
112,2189,the game allows the use of Wii mote only,5


In [50]:
mae_df = precision_prevalence.merge(recall_prevalence)
mae_df

,id,key_point,correct_prevalence,expected_prevalence
0,3609,the old key components need to be taken to a l...,5,5
1,2799,you need two originally programmed keys to pro...,16,18
2,2799,it's possible to use a remake key or a key fro...,5,14
3,2501,using a detailer as a lubricant,2,2
4,3250,the adjustable cup holders are easy to use and...,12,12
...,...,...,...,...
109,539,the 3DS is backwards compatible with DS games,6,6
110,1837,Infamous Second Son is a bigger and more refin...,11,13
111,1837,Infamous Second Son is a game that surpassed t...,4,12
112,2189,the game allows the use of Wii mote only,5,5


In [51]:
mae_df.apply(lambda x: abs(x['correct_prevalence'] - x['expected_prevalence']), axis=1).mean()

4.2368421052631575